In [1]:
from pymongo import MongoClient

local = False
if local:
    client = MongoClient('fc8iasm01', 27017)
    plan_col = client.aca.plan
    drug_col = client.aca.drug
else:
    client = MongoClient('ec2-52-53-230-141.us-west-1.compute.amazonaws.com', 27017)
    plan_col = client.plans.plans
    drug_col = client.formularies.drugs
    prov_col = client.providers.providers
    faci_col = client.providers.facilities

all_plan = drug_col.distinct('plans.plan_id')
all_drug = drug_col.distinct('rxnorm_id')

print 'using %s Mongo, total drug: %d, total plan: %d, total provider: %d' %(
     'local' if local else 'aws', len(all_drug), len(all_plan), prov_col.count())

using aws Mongo, total drug: 46206, total plan: 6035, total provider: 8799098


In [2]:
import numpy as np
state_ids = np.unique([i[5:7] for i in all_plan])
print 'find plan from %d states: %s' %(len(state_ids), ', '.join(state_ids))

find plan from 43 states: AK, AL, AR, AZ, CO, DE, FL, GA, HI, IA, IL, IN, KS, KY, LA, MA, ME, MI, MN, MO, MS, MT, NC, ND, NE, NH, NJ, NM, NV, OH, OK, OR, PA, SC, SD, TN, TX, UT, VA, WA, WI, WV, WY


In [22]:
%reload_ext autoreload
%autoreload 2
from extract_plan_feature import *
from extract_drug_feature import *
from extract_provider_feature import *
from get_state_feature import *

In [5]:
state_plan = [i for i in all_plan if 'SD' in i]

In [15]:
p_fea = extract_plan_feature(plan_col, state_plan)

get formulary state space for all plans
total formulary states: 12
extract formulary states for each plan
complete for 18 plans
get formulary summary feature for each plan


In [20]:
d_fea = extract_drug_feature(drug_col, state_plan)

get all drugs covered by all plans
total rx: 6486
check drug coverage for each plan
complete for 43 plans
get summary feature for drug
total drug states: 28
extract drug sumstat for each plan
complete for 43 plans


In [23]:
v_fea = extract_provider_feature(prov_col, state_plan)

get provider under the plans
total providers: 18884
check provider coverage for each plan
complete for 43 plans
get summary feature for provider
total provider summary: 1318
extract provider sumstat for each plan
complete for 43 plans


In [32]:
# fea_mat=p_fea+v_fea
from sets import Set
valid_plan = Set(fea_mat[0].keys())
for i in range(1, len(fea_mat)):
    valid_plan = valid_plan.intersection(fea_mat[i].keys())
valid_plan = list(valid_plan)

In [43]:
plan,feature = get_state_feature(state_plan, plan_col, drug_col, prov_col)
# temp=[hstack([f[p] for f in fea_mat]) for p in valid_plan]

get formulary state space for all plans
total formulary states: 12
extract formulary states for each plan
complete for 18 plans
get formulary summary feature for each plan
get all drugs covered by all plans
total rx: 6486
check drug coverage for each plan
complete for 43 plans
get summary feature for drug
total drug states: 28
extract drug sumstat for each plan
complete for 43 plans
get provider under the plans
total providers: 18884
check provider coverage for each plan
complete for 43 plans
get summary feature for provider
total provider summary: 1318
extract provider sumstat for each plan
complete for 43 plans


In [49]:
save_name = 'feature/%s_%d_%d.pickle' %('SD', feature.shape[0], feature.shape[1])

In [51]:
import pickle
with open(save_name, 'w') as f:
    pickle.dump([feature, plan], f)

In [53]:
from s3_helpers import *
s3_helper().upload(save_name)